# Neupy Tutorial

### ref: http://neupy.com/2016/11/12/mnist_classification.html#mnist-classification

In [1]:
%%bash
git add Neupy_Tutorial.ipynb
git commit -m 'Init tutorial file'
git push

On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
	modified:   Handwritten_Neural.ipynb

Untracked files:
	.ipynb_checkpoints/
	Neupy_Tutorial.ipynb
	test.csv
	train.csv

no changes added to commit


fatal: pathspec 'Neupy_Tutorial.ipynbg' did not match any files
Everything up-to-date


In [ ]:
from sklearn import datasets, model_selection
mnist = datasets.fetch_mldata('MNIST original')
data, target = mnist.data, mnist.target

In [ ]:
data.head()